In [ ]:
from vosk import Model, KaldiRecognizer, SpkModel
import sys
import wave
import json
import os
import numpy as np
import IPython
from vosk import SetLogLevel
import warnings
from tkinter import *  
import sys
import time
from transformers import logging
from recasepunc import CasePuncPredictor
from recasepunc import WordpieceTokenizer
from recasepunc import Config
import tkinter.filedialog as fd
from tkinter import messagebox as mb
warnings.filterwarnings('ignore')
SetLogLevel(-1)



logging.set_verbosity_error()
predictor = CasePuncPredictor('checkpoint', lang="ru")

model_path = "/Users/daniilkobelev/Downloads/vosk-model-ru-0.22 2"
spk_model_path = "/Users/daniilkobelev/Downloads/vosk-model-spk-0.4"

model = Model(model_path)
spk_model = SpkModel(spk_model_path)

In [ ]:
def punc(text, final):
    tokens = list(enumerate(predictor.tokenize(text)))
    results = ""
    respunc = ''
    for token, case_label, punc_label in predictor.predict(tokens, lambda x: x[1]):
        prediction = predictor.map_punc_label(predictor.map_case_label(token[1], case_label), punc_label)
        if token[1][0] != '#':
            results = results + ' ' + prediction
        else:
            results = results + prediction
    respunc = respunc + results
    final += '-' + respunc + '\n'
    return final

def choose_file():  
    wavfile = fd.askopenfilename()   
    
def clicked():  
    wf = wave.open(wavfile, "rb") 
    rec = KaldiRecognizer(model, wf.getframerate() * wf.getnchannels(), spk_model)
    rec.SetSpkModel(spk_model)
    wf.rewind()
    final = ''
    while True:    
        data = wf.readframes(4000)
        datalen=len(data);
        if datalen == 0:
            res = json.loads(rec.FinalResult())
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            final = punc(res['text'], final)
        if datalen == 0:
            break
    mb.showinfo(title='Распознанный текст', message=final)
        
from tkinter import *
from tkinter.filedialog import *
import fileinput
root = Tk()
root.title("МОИ задание №2")
root.minsize(width=500, height=400)
btn = Button(root, text="Выберите файл(*.WAV)", command=choose_file)
btn.pack(padx=120, pady=30)
btn1 = Button(root, text="Подтвердить", command=clicked)
btn1.pack(padx=150, pady=30)

root.mainloop()